# View Fors2 , View spectra one by one and Compare to SL and write in a pdf file

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNRS
- creation date : 2023/01/21
- last update : 2023/01/21
- FORS2 : https://github.com/ericnuss/fors2
- Result of StarLight : /sps/lsst/groups/photoz/fors2/FORS2/ResStarlight



          /Users/dagoret/MacOSX/GitHub/LSST/FORS2>ls -l
              total 0
              drwxrwsr-x  3 dagoret  staff   96 Aug 21  2018 ResStarlight
              drwxr-xr-x  9 dagoret  staff  288 Dec 19 18:01 fors2

## Import

In [ ]:
import os,sys,re
from astropy.io import fits
from astropy.table import Table

In [ ]:
# Import some generally useful packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx

from matplotlib.backends.backend_pdf import PdfPages

%matplotlib inline
import pandas as pd
from itertools import cycle, islice


In [ ]:
from scipy import ndimage

#from scipy import interp as scinterp
from scipy.interpolate import interp1d

import glob
import collections
#from def_raw_seds import *

In [ ]:
# to enlarge the sizes
params = {'legend.fontsize': 'large',
          'figure.figsize': (8, 4),
         'axes.labelsize': 'xx-large',
         'axes.titlesize':'xx-large',
         'xtick.labelsize':'xx-large',
         'ytick.labelsize':'xx-large'}
plt.rcParams.update(params)

In [ ]:
import matplotlib.offsetbox
props = dict(boxstyle='round',edgecolor="w",facecolor="w", alpha=0.5)

In [ ]:
from def_raw_seds import *

In [ ]:
from raw_data_analysis import *

In [ ]:
# Import the primary photometry classes from rubin_sim.photUtils

import rubin_sim.photUtils.Bandpass as Bandpass
import rubin_sim.photUtils.Sed as Sed
from rubin_sim.data import get_data_dir

In [ ]:
pysynphot_root_path=os.environ['PYSYN_CDBS']
path_sed_calspec=os.path.join(pysynphot_root_path,'calspec')
# pysynphot
import pysynphot as S
S.refs.setref(area=1)
S.refs.getref()

In [ ]:
from astropy.io import fits

In [ ]:
from scipy.ndimage import gaussian_filter1d

In [ ]:
from GalaxyLineDict import build_emissionlinesdict

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
import copy

In [ ]:
def SaveImageInPdf(all_figs,outputfile):
    """
    """
    with PdfPages(outputfile) as pdf:
        for fig in all_figs:
            pdf.savefig(fig)
            plt.close()

# Start

In [ ]:
cwd=os.path.abspath("")
main_dir = cwd
print(f"notebook current executing path : {cwd}")

## Configuration

### Emission lines

In [ ]:
#df_lines=pd.read_excel("datatools/GalEmissionLines.xlsx")

In [ ]:
#df_lines

In [ ]:
#df_sdss_lines = pd.read_excel("datatools/sdss_galaxylines.xlsx")

In [ ]:
#df_sdss_lines

# Lines

In [ ]:
lines_to_plot=build_emissionlinesdict()

In [ ]:
lines_to_plot

In [ ]:
def plot_line(ax,line_name):
    
    #keys = lines_to_plot.keys()
    #for the_key in keys:
    #    if the_key in line_name:
    #        print("the_key=",the_key," line_name=",line_name)
    #        line_name=the_key
    #        break
            
    posx0=0.0
    posy0=0.05
    fontsize=12
    the_color = {"emission":'r',"absorption":"g","break":"purple"}
    the_line_to_plot = lines_to_plot[line_name]
    wls = the_line_to_plot["wls"]
    tag = the_line_to_plot["name"]
    typ = the_line_to_plot["type"]
    
    xlim = ax.get_xlim()
    for wl in wls:
        ax.axvline(wl, lw=2, color=the_color[typ])
        xpos = (wl - xlim[0]) / (xlim[1] - xlim[0])+posx0 
        if 0 < xpos < 1:
            ax.annotate(tag, xy=(xpos, posy0), rotation=0, ha='left', va='bottom',
                            xycoords='axes fraction', color=the_color[typ], fontsize=fontsize)
    return ax

## StarLight

In [ ]:
filename = 'datatools/spectra_SL_data.fits'
hdu = fits.open(filename)
data_in=hdu[0].data
hdr_in = hdu[0].header

In [ ]:
NSED_SL = data_in.shape[0]-3
NWL_SL = data_in.shape[1]-1

In [ ]:
wavelengths_SL = data_in[0,1:] 
flambdas_average_SL = data_in[1,1:] 
flambdas_std_SL = data_in[2,1:] 
spec_SL = data_in[3:,1:] 
all_numbers_SL = data_in[3:,0] 

### Fors2

## Start

### Catalog

In [ ]:
t = Table.read(filename_fits_catalog)

In [ ]:
t

### List

In [ ]:
lst=fors2_jct_SL() #remet les spectres dans le rest frame (ie a z=0)

In [ ]:
lst = sorted(lst)
print(lst)

In [ ]:
all_specfilenames = os.listdir(path_out_jct)

In [ ]:
all_specfilenames = sorted(all_specfilenames)

In [ ]:
all_numbers = [ int(re.findall('^SPEC(.*)[.]txt$',fn)[0]) for fn in all_specfilenames ]

In [ ]:
all_numbers = sorted(all_numbers)

In [ ]:
print(all_numbers)

In [ ]:
NSED = len(all_numbers)

# wavelength bin colors
jet = plt.get_cmap('jet')
cNorm = colors.Normalize(vmin=0, vmax=NSED)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
all_colors = scalarMap.to_rgba(np.arange(NSED), alpha=1)

In [ ]:
cut = t["ID"] == 3
t[cut]

In [ ]:
all_spectra = []
all_wl = []
all_tag = []
all_lines = []
#all_sed = []

for the_num in all_numbers:
    tcut = t["ID"] == the_num
    redshift = t[tcut]["z"].value[0]
    lines = t[tcut]["Lines"].value[0]
    filename = f"SPEC{the_num}.txt"
    tag = f"SPEC{the_num} , z={redshift:.2f}, lines={lines}"
    fullfilename = os.path.join(path_out_jct,filename)

    arr = np.loadtxt(fullfilename)
    all_wl.append(arr[:,0])
    all_spectra.append(arr[:,1])
    all_tag.append(tag)
    all_lines.append(lines)
 

In [ ]:
if 0:
    for idx in range(NSED):
        plt.figure(figsize=(16,2))
        plt.plot(all_wl[idx],all_spectra[idx], color=all_colors[idx],label=all_tag[idx])
        plt.legend(loc="upper right")
        plt.grid()
        plt.axvline(4150,color="k")
        plt.axvline(4250,color="k")
#    plt.ylim(-0.02,0.4)
        plt.xlim(2000.,8000.)
        plt.show()


In [ ]:
idx=0
the_lines = all_lines[idx]
print("the_lines = ",the_lines)

all_elements  = the_lines.decode().split(",")


fig, axs = plt.subplots(1, 1, figsize=(16, 8))
ax=axs

the_wl_fors2 = all_wl[idx]
the_flux_fors2 = all_spectra[idx]

the_max_flux_1 = the_wl_fors2.max()

ax.plot(the_wl_fors2 ,the_flux_fors2 ,'-' ,color=all_colors[idx],label=all_tag[idx],lw=3)

the_flux_fors2_filt  = gaussian_filter1d(the_flux_fors2,40)
ax.plot(the_wl_fors2 ,the_flux_fors2_filt ,'-' ,color='yellow',lw=3)

wlmin = the_wl_fors2.min()
wlmax = the_wl_fors2.max()

I1=np.trapz(the_flux_fors2_filt ,the_wl_fors2)

ax.legend(loc="upper right")
ax.grid()

wl1=4150
wl2=4250

#ax.axvline(wl1,color="grey")
#ax.axvline(wl2,color="grey")




#    plt.ylim(-0.02,0.4)
ax.set_xlim(0.,10000.)
for el in all_elements:
    el=el.split(" ")[0]  
    print(el)
    if len(el)>0:
        plot_line(ax,el)

        
the_num = all_numbers[idx]

idx_sl=np.where(all_numbers_SL==the_num)[0][0]
the_spectrum_SL = spec_SL[idx_sl]

the_max_flux_2 = the_spectrum_SL.max()

wavelengths_SL_indexes=np.where(np.logical_and(wavelengths_SL>=wlmin,wavelengths_SL<wlmax))[0]

the_spectrum_SL_flux_cut = the_spectrum_SL[wavelengths_SL_indexes]
the_spectrum_SL_wl_cut = wavelengths_SL[wavelengths_SL_indexes]

I2=np.trapz(the_spectrum_SL_flux_cut,the_spectrum_SL_wl_cut)

g=I1/I2

the_flux_SL_toplot= g*spec_SL[idx_sl]

the_max_flux_2=the_flux_SL_toplot.max()

ymax=1.1*max(the_max_flux_1,the_max_flux_2)

ax.plot(wavelengths_SL,the_flux_SL_toplot,color="orange")

ymin=0

#ax.set_ylim(ymin,ymax)

## Write in pdf File

In [ ]:
NFIGSPERPAGE = 8
NCOLSPERPAGE = 1
NROWSPERPAGE = int(np.ceil(NFIGSPERPAGE/NCOLSPERPAGE)) 

In [ ]:
all_figs = []
idx_page=0

for idx in range(0,NSED):
    
    if idx%NFIGSPERPAGE == 0:
        idx_page+=1
        
        plt.tight_layout()
        plt.show()
        # save previous page
        if idx_page>1:
            all_figs.append(copy.deepcopy(fig))
        
        # create a new page
        fig=plt.figure(figsize=(16,30))
        idx_fig_inpage=0
        
    #increase fig counter in mage
    idx_fig_inpage+=1    
        
    ax=fig.add_subplot(NROWSPERPAGE,NCOLSPERPAGE,idx_fig_inpage)
    
    
    the_tag=all_tag[idx] 
    print(f"===================== SED {idx} : {the_tag}=======================")
    
    the_lines = all_lines[idx]
    print("the_lines = ",the_lines)
    all_elements  = the_lines.decode().split(",")
    
    
    the_wl_fors2 = all_wl[idx]
    the_flux_fors2 = all_spectra[idx]
    ymax1 = the_flux_fors2.max()
    
    
    #ax.plot(the_wl_fors2,the_flux_fors2,'-' ,color=all_colors[idx],label=all_tag[idx],lw=4)
    ax.plot(the_wl_fors2,the_flux_fors2,'-' ,color='b',label=all_tag[idx],lw=4)
    
    the_flux_fors2_filt  = gaussian_filter1d(the_flux_fors2,40)
    #ax.plot(the_wl_fors2 ,the_flux_fors2_filt ,'-' ,color='yellow',lw=3)

    wlmin = the_wl_fors2.min()
    wlmax = the_wl_fors2.max()

    I1=np.trapz(the_flux_fors2_filt,the_wl_fors2)
    
    
    ax.legend(loc="upper right")
    ax.grid()
    
    wl1=4150
    wl2=4250

#    ax.axvline(wl1,color="grey")
#    ax.axvline(wl2,color="grey")
#    plt.ylim(-0.02,0.4)

    ax.set_xlim(0.,10000.)
    for el in all_elements:
        el=el.split(" ")[0]  
        el=el.split("\r")[0]  
        print(el)
        if len(el)>0:
            plot_line(ax,el)
            
            
    the_num = all_numbers[idx]
    print(f"the_num = {the_num}")

    idx_sl=np.where(all_numbers_SL==the_num)[0][0]
    the_spectrum_SL = spec_SL[idx_sl]

    wavelengths_SL_indexes=np.where(np.logical_and(wavelengths_SL>=wlmin,wavelengths_SL<wlmax))[0]

    the_spectrum_SL_flux_cut = the_spectrum_SL[wavelengths_SL_indexes]
    the_spectrum_SL_wl_cut = wavelengths_SL[wavelengths_SL_indexes]

    I2=np.trapz(the_spectrum_SL_flux_cut,the_spectrum_SL_wl_cut)

    g=I1/I2
    
    the_spectrum_SL_toplot = g*spec_SL[idx_sl]
    
    ymax2 = the_spectrum_SL_toplot.max()
    
    ymax=1.1*max(ymax1,ymax2)

    ax.plot(wavelengths_SL,the_spectrum_SL_toplot,color="orange",label="SL")
    ax.set_ylim(0,ymax)
    
    
    

In [ ]:
pdfoutputfilename = f"multipage_pdf_CompareFors2andSLSpectra.pdf"
SaveImageInPdf(all_figs,pdfoutputfilename)